In [39]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [40]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-SEARCH")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-SEARCH-ADVANCED


In [41]:
# 라이브러리 선언
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever
import re


In [42]:
# 1. 임베딩(Embedding) 생성

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 1536 차원

# dimensions 파라미터 설정 (원하는 차원 수로 축소)
# embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     dimensions=512  # 512 차원으로 축소 (최대 1536까지 가능)
# )

In [63]:
def print_result(query, result):
    print(f"'{query}'와 유사한 상품 검색 결과:")
    print("-" * 60)

    if result:
        for i, (doc) in enumerate(result, 1):
            print(f"[결과 {i}]")
            print(f"{doc.page_content}")
            # print(f"메타데이터: {doc.metadata}")  # 모든 메타데이터 출력
            try:
                print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
                print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
                print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
                print(f"- 품목정보: {doc.metadata.get('ARTC_INFO', '정보 없음')}")
                print(f"- 카테고리: {doc.metadata.get('CATEGORY_NMS', '정보 없음')}")
                print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
                print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
                print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
                print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
                print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
                print(f"- 주요 특징 및 기능:")
                feature_values = doc.metadata['OPT_VAL_DESC'].split(',')
                feature_titles = doc.metadata['OPT_DISP_NM'].split(',')
                for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                    print(f"  - {title} : {value}")
                    if i < 3: break
                print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
            except Exception as e:
                print(f"메타데이터 처리 중 오류 발생: {str(e)}")
            print("-" * 60)
    else:
        print("검색 결과가 없습니다.")
        print("다른 검색어로 다시 시도해보시거나, 검색어를 더 구체적으로 입력해주세요.")

In [43]:
# 2. 의도 기반 필터링 함수 선언

def intent_based_filtering(query):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격필터
    filter_dict.update(intent_based_price_filtering(query))

    # 브랜드필터

    # 속성필터

    return filter_dict

def intent_based_price_filtering(query):
    """사용자 의도 분석을 통한 필터링"""

    filter_dict = {}

    # 가격 필터링을 위한 정규표현식 패턴
    # 1) N만원 이상 | N만원 부터
    price_pattern = r'(\d+)만원\s*(이상|부터)'
    price_match = re.search(price_pattern, query)
    
    if price_match:
        # SALE_PRC 키가 없으면 생성
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        # 만원 단위를 원 단위로 변환 (예: 120만원 -> 1200000원)
        filter_dict["SALE_PRC"]["$gte"] = int(price_match.group(1)) * 10000
    
    # 2) N만원 이하 | N만원 까지
    price_pattern_lte = r'(\d+)만원\s*(이하|까지)'
    price_match_lte = re.search(price_pattern_lte, query)
    
    if price_match_lte:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        filter_dict["SALE_PRC"]["$lte"] = int(price_match_lte.group(1)) * 10000

    # 3) N만원 대
    price_pattern_range = r'(\d+)만원\s*대'
    price_match_range = re.search(price_pattern_range, query)
    
    if price_match_range:
        if "SALE_PRC" not in filter_dict:
            filter_dict["SALE_PRC"] = {}
        base_price = int(price_match_range.group(1)) * 10000
        filter_dict["SALE_PRC"]["$gte"] = base_price
        filter_dict["SALE_PRC"]["$lt"] = base_price * 2

    return filter_dict


In [44]:
# DB 경로 설정
persist_directory = "../.db/faiss"

vectorstore = FAISS.load_local(
    persist_directory, 
    embeddings, 
    allow_dangerous_deserialization=True)

**매개변수(parameters)**

- `**kwargs`: 검색 함수에 전달할 키워드 인자
  - `search_type`: 검색 유형 ("similarity", "mmr", "similarity_score_threshold")
  - `search_kwargs`: 추가 검색 옵션
    - `k`: 반환할 문서 수 (기본값: 4)
    - `score_threshold`: similarity_score_threshold 검색의 최소 유사도 임계값
    - `fetch_k`: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
    - `lambda_mult`: MMR 결과의 다양성 조절 (0-1 사이, 기본값: 0.5)
    - `filter`: 문서 메타데이터 기반 필터링

In [59]:
from langchain_core.runnables import ConfigurableField

# k 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 1}).configurable_fields(
    search_type=ConfigurableField(
        id="search_type",
        name="Search Type",
        description="The search type to use",
    ),
    search_kwargs=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정
        id="search_kwargs",
        # 검색 매개변수의 이름을 설정
        name="Search Kwargs",
        # 검색 매개변수에 대한 설명을 작성
        description="The search kwargs to use",
    ),
)

In [68]:

# 검색할 쿼리 설정 (원하는 검색어로 변경 가능)
query = "100만원 이상 삼성전자 TV"  # 예시 쿼리: 원하는 검색어로 변경하세요

# 필터링 옵션 입력 받기
filter_dict = intent_based_filtering(query)
# filter_dict = {"GOODS_NO": '0031055945'}
# filter_dict = {
#     "SALE_PRC": {
#         "$gte": 1000000,  # 최소 가격
#         "$lte": 2000000   # 최대 가격
#     }
# }

# 검색어에 따라 적절한 설정을 지정
config = {
    "configurable": {
        "search_type": "similarity_score_threshold", # similarity, similarity_score_threshold, mmr
        "search_kwargs": {
            "score_threshold": 0.4,
            # "k": 4,
            "filter": filter_dict
        },
    }
}

result = retriever.invoke(query, config=config)

print(f"필터: {filter_dict}")

print_result(query, result)

필터: {'SALE_PRC': {'$gte': 1000000}}
'100만원 이상 삼성전자 TV'와 유사한 상품 검색 결과:
------------------------------------------------------------
[결과 1]
삼성전자의 LED TV, LH75BECHLGF 189.3Cm(75) 비지니스 TV [설치유형 선택가능]. 카테고리: TV·영상가전 > TV > 대형(~189cm). 가격 정보: 판매가 1,179,000원, 롯데카드 카드 사용 시 7% 할인되어 최대 1,096,470원. 가전제품입니다.
- 상품상태: 정상상품
- 브랜드명: 삼성전자
- 상품명: LH75BECHLGF 189.3Cm(75) 비지니스 TV [설치유형 선택가능]
- 품목정보: TV>LED TV
- 카테고리: TV·영상가전>TV>대형(~189cm)
- 판매가: 1,179,000원
- 할인가: 1,179,000원
- 최대혜택가: 1,096,470원
- 카드할인율: 7%
- 할인카드: 롯데카드,현대카드,네이버페이,삼성카드,신한카드,KB국민카드
- 주요 특징 및 기능:
  -  : 
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0020639642
------------------------------------------------------------
[결과 2]
삼성전자의 LED TV, 삼성전자 LH75BEDHLGF 189Cm 4K UHD HDR10+ 1등급 비지니스 TV (설치유형 선택가능). 카테고리: TV·영상가전 > TV > 대형(~189cm). 가격 정보: 판매가 1,179,000원, 롯데카드 카드 사용 시 7% 할인되어 최대 1,096,470원. 가전제품입니다.
- 상품상태: 정상상품
- 브랜드명: 삼성전자
- 상품명: 삼성전자 LH75BEDHLGF 189Cm 4K UHD HDR10+ 1등급 비지니스 TV (설치유형 선택가능)
- 품목정보: TV>LED TV
- 카테고리: TV

### `EmbeddingsFilter`

각각의 검색된 문서에 대해 추가적인 LLM 호출을 수행하는 것은 비용이 많이 들고 속도가 느립니다.

`EmbeddingsFilter`는 문서와 쿼리를 임베딩하고 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환함으로써 더 저렴하고 빠른 옵션을 제공합니다.

이를 통해 검색 결과의 관련성을 유지하면서도 계산 비용과 시간을 절약할 수 있습니다.


In [67]:
# 유사도 임계값이 0.4인 EmbeddingsFilter 객체를 생성합니다.
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.4)

# 기본 압축기로 embeddings_filter를, 기본 검색기로 retriever를 사용하여 ContextualCompressionRetriever 객체를 생성합니다.
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

# ContextualCompressionRetriever 객체를 사용하여 관련 문서를 검색합니다.
compressed_docs = compression_retriever.invoke(
    # 쿼리
    query,
    config=config
)
# 압축된 문서를 예쁘게 출력합니다.
print_result(query, compressed_docs)

'100만원 이상 삼성전자 TV'와 유사한 상품 검색 결과:
------------------------------------------------------------
[결과 1]
삼성전자의 LED TV, LH75BECHLGF 189.3Cm(75) 비지니스 TV [설치유형 선택가능]. 카테고리: TV·영상가전 > TV > 대형(~189cm). 가격 정보: 판매가 1,179,000원, 롯데카드 카드 사용 시 7% 할인되어 최대 1,096,470원. 가전제품입니다.
- 상품상태: 정상상품
- 브랜드명: 삼성전자
- 상품명: LH75BECHLGF 189.3Cm(75) 비지니스 TV [설치유형 선택가능]
- 품목정보: TV>LED TV
- 카테고리: TV·영상가전>TV>대형(~189cm)
- 판매가: 1,179,000원
- 할인가: 1,179,000원
- 최대혜택가: 1,096,470원
- 카드할인율: 7%
- 할인카드: 롯데카드,현대카드,네이버페이,삼성카드,신한카드,KB국민카드
- 주요 특징 및 기능:
  -  : 
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0020639642
------------------------------------------------------------


In [14]:
# 쿼리와 유사한 상품 5개 검색
results_with_score = vectorstore.similarity_search_with_score(
    query=query, 
    filter = filter_dict, # 메타데이터 필터 적용
    k=4  # 반환할 결과 수
)

# 결과 출력
print(f"'{query}'와 유사한 상품 검색 결과 (최대 4개):")
print("-" * 60)

if results_with_score:
    for i, (doc, score) in enumerate(results_with_score, 1):
        print(f"[결과 {i}] 유사도: {score}") # 낮을수록 더 유사함
        print(f"{doc.page_content}")
        # print(f"메타데이터: {doc.metadata}")  # 모든 메타데이터 출력
        try:
            print(f"- 상품상태: {doc.metadata.get('GOODS_STAT_SCT_NM', '정보 없음')}")
            print(f"- 브랜드명: {doc.metadata.get('BRND_NM', '정보 없음')}")
            print(f"- 상품명: {doc.metadata.get('GOODS_NM', '정보 없음')}")
            print(f"- 품목정보: {doc.metadata.get('ARTC_INFO', '정보 없음')}")
            print(f"- 카테고리: {doc.metadata.get('CATEGORY_NMS', '정보 없음')}")
            print(f"- 판매가: {format(int(doc.metadata.get('SALE_PRC', 0)), ',')}원")
            print(f"- 할인가: {format(int(doc.metadata.get('DSCNT_SALE_PRC', 0)), ',')}원")
            print(f"- 최대혜택가: {format(int(doc.metadata.get('MAX_BENEFIT_PRICE', 0)), ',')}원")
            print(f"- 카드할인율: {doc.metadata.get('CARD_DC_RATE', '0')}%")
            print(f"- 할인카드: {doc.metadata.get('CARD_DC_NAME_LIST', '정보 없음')}")
            print(f"- 주요 특징 및 기능:")
            feature_values = doc.metadata['OPT_VAL_DESC'].split(',')
            feature_titles = doc.metadata['OPT_DISP_NM'].split(',')
            for i, (title, value) in enumerate(zip(feature_titles, feature_values)):
                print(f"  - {title} : {value}")
                if i < 3: break
            print(f"🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo={doc.metadata.get('GOODS_NO', '정보 없음')}")
        except Exception as e:
            print(f"메타데이터 처리 중 오류 발생: {str(e)}")
        print("-" * 60)
else:
    print("검색 결과가 없습니다.")
    print("다른 검색어로 다시 시도해보시거나, 검색어를 더 구체적으로 입력해주세요.")


'100만원 이하 LG전자 TV'와 유사한 상품 검색 결과 (최대 4개):
------------------------------------------------------------
[결과 1] 유사도: 0.804122269153595
LG전자의 UHD TV, 107cm LG전자 LED TV 43UT8300NNA (설치유형 선택가능). 카테고리: TV·영상가전 > TV > 중소형(~109cm). 가격 정보: 판매가 640,000원, 롯데카드 카드 사용 시 10% 할인되어 최대 576,000원. 가전제품입니다.
- 상품상태: 정상상품
- 브랜드명: LG전자
- 상품명: 107cm LG전자 LED TV 43UT8300NNA (설치유형 선택가능)
- 품목정보: TV>UHD TV
- 카테고리: TV·영상가전>TV>중소형(~109cm)
- 판매가: 640,000원
- 할인가: 640,000원
- 최대혜택가: 576,000원
- 카드할인율: 10%
- 할인카드: 롯데카드,롯데카드,현대카드,네이버페이,삼성카드,신한카드,KB국민카드
- 주요 특징 및 기능:
  -  : 
🔗 상품보러가기 : https://www.e-himart.co.kr/app/goods/goodsDetail?goodsNo=0040298827
------------------------------------------------------------
[결과 2] 유사도: 0.8181734085083008
LG전자의 LED TV, 125cm 울트라 HD TV 50UT8300ENA 벽걸이형. 주요 특징: 크기: 50~59인치, 종류: LED, 해상도: UHD(4K), 기본 주사율: 60Hz, 스마트기능: 넷플릭스, 유튜브, 에어플레이2, 부가기능: USB재생, 부가기능: 블루투스5.0, 부가기능: 게임모드, 부가기능: HDMI eARC, e효율등급: 1등급, 출시년도: 2024년, 화면 타입: 평면형, 스피커 출력: 20W, 음향효과: 공간인식사운드, 음향효과: 인공지능사운드, 지원단자: HDMI, 지원단자: